## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [16]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras import regularizers
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5425840326990696426
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4963368960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2880440103983095220
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [17]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [18]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.fit_transform(y_test).toarray()

In [25]:
classifier=Sequential()
#卷積組合
classifier.add(Convolution2D(filters = 32 ,kernel_size=(3,3) ,strides = 1 ,padding = 'same',input_shape=(32,32,3),activation='relu'))
classifier.add(MaxPooling2D(2,2))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters = 64 ,kernel_size=(3,3) ,strides = 1 ,padding = 'same',activation = 'relu'))
classifier.add(MaxPooling2D(2,2))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(filters = 32 ,kernel_size=(3,3) ,strides = 1 ,padding = 'same',activation = 'relu'))
classifier.add(MaxPooling2D(2,2))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())
#FC
classifier.add(Dense(100,activation = "relu",kernel_regularizer=regularizers.l2(l=0.001))) 
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())
classifier.add(Dense(100,activation = "relu",kernel_regularizer=regularizers.l2(l=0.001)))
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())
classifier.add(Dense(100,activation = "relu",kernel_regularizer=regularizers.l2(l=0.001)))
classifier.add(Dropout(0.5))
classifier.add(BatchNormalization())



#輸出
classifier.add(Dense(10,activation='softmax'))

In [26]:
image_ge = ImageDataGenerator(
 featurewise_center=True,
 featurewise_std_normalization=True,
 rotation_range=10,
 width_shift_range=0.1,
 height_shift_range=0.1,
 shear_range=0.1,
 zoom_range=0.1,
 horizontal_flip=True,
 vertical_flip=False,
 dtype=np.float32
)

image_ge.fit(x_train)

In [24]:
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = classifier.fit(image_ge.flow(x_train,y_train,batch_size = 100),epochs=50,validation_data=(x_test,y_test))

Epoch 1/50
500/500 [==============================] - 19s 37ms/step - loss: 2.1125 - accuracy: 0.3466 - val_loss: 1.6845 - val_accuracy: 0.4740
Epoch 2/50
500/500 [==============================] - 19s 38ms/step - loss: 1.6736 - accuracy: 0.4766 - val_loss: 1.4221 - val_accuracy: 0.5615
Epoch 3/50
500/500 [==============================] - 18s 36ms/step - loss: 1.5294 - accuracy: 0.5243 - val_loss: 1.3194 - val_accuracy: 0.5906
Epoch 4/50
500/500 [==============================] - 18s 36ms/step - loss: 1.4422 - accuracy: 0.5522 - val_loss: 1.1909 - val_accuracy: 0.6432
Epoch 5/50
500/500 [==============================] - 19s 39ms/step - loss: 1.3828 - accuracy: 0.5731 - val_loss: 1.2786 - val_accuracy: 0.6018
Epoch 6/50
500/500 [==============================] - 18s 36ms/step - loss: 1.3485 - accuracy: 0.5895 - val_loss: 1.1869 - val_accuracy: 0.6418
Epoch 7/50
500/500 [==============================] - 18s 36ms/step - loss: 1.3191 - accuracy: 0.6029 - val_loss: 1.2295 - val_accuracy:

KeyboardInterrupt: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [23]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[2.55108532e-03, 1.84481469e-05, 1.10352754e-04, 9.01590884e-01,
        1.26449252e-02, 1.77490890e-07, 7.37183000e-05, 7.02636462e-05,
        8.29149187e-02, 2.51464389e-05]], dtype=float32)

In [1]:
import pandas as pd 
from matplotlib import pyplot as plt

pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(-0.1,1.5)
plt.show()

NameError: name 'history' is not defined